# **Sprint04: Detecção de Cáries / Análise de Imagens Odontológicas Panoramic Dental Dataset**

Este notebook demonstra o processo de:
1. Instalação e configuração da API do Kaggle no Google Colab.
2. Download e descompactação de um dataset de imagens (Panoramic Dental Dataset).
3. Carregamento e pré-processamento dos dados.
4. Construção, compilação e treinamento de um modelo de rede neural.
5. Avaliação e análise dos resultados obtidos.

As melhorias em relação a versões anteriores incluem uma melhor organização do código, adição de comentários explicativos e sugestões de extensões (como callbacks, visualização de métricas e análise de erros).



## 1. Instalação da Biblioteca Kaggle

Nesta etapa, instalamos a biblioteca `kaggle` no ambiente do Google Colab. Isso permite interagir diretamente com o Kaggle para baixar datasets, enviar submissões de competições, etc.


In [2]:

!pip install kaggle

## 2. Upload da Chave da API do Kaggle

Aqui fazemos o upload do arquivo `kaggle.json`, que contém as credenciais de acesso à sua conta Kaggle.  


In [1]:

from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"pedrolpsp","key":"1d2ef680745fcd4960d1dd4079f50218"}'}

## 3. Configuração do Diretório do Kaggle

Após o upload, precisamos mover o `kaggle.json` para o diretório padrão `~/.kaggle/` e ajustar as permissões para que a biblioteca Kaggle possa acessá-lo corretamente.


In [3]:

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

## 4. Download do Dataset

Agora podemos baixar o dataset escolhido no Kaggle.  
**Exemplo**: `thunderpede/panoramic-dental-dataset`.


In [4]:

!kaggle datasets download -d  thunderpede/panoramic-dental-dataset

Dataset URL: https://www.kaggle.com/datasets/thunderpede/panoramic-dental-dataset
License(s): other
 75% 182M/244M [00:00<00:00, 595MB/s] 
100% 244M/244M [00:00<00:00, 565MB/s]


## 5. Descompactar o Dataset

Por fim, descompactamos o arquivo `.zip` baixado para obtermos as pastas e arquivos de imagem.


In [7]:

!unzip panoramic-dental-dataset.zip

Archive:  panoramic-dental-dataset.zip
replace annotations/bboxes_caries/1008.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: annotations/bboxes_caries/1008.txt  
replace annotations/bboxes_caries/1009.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace annotations/bboxes_caries/1009.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: annotations/bboxes_caries/1009.txt  
replace annotations/bboxes_caries/1016.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace annotations/bboxes_caries/1016.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace annotations/bboxes_caries/1016.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace annotations/bboxes_caries/1016.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace annotations/bboxes_caries/1016.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: annotations/bboxes_caries/1016.

In [16]:
import os
import shutil

# Criar pastas de classes se não existirem
os.makedirs('/content/images/carie', exist_ok=True)
os.makedirs('/content/images/saudavel', exist_ok=True)

# Pasta de origem das imagens desorganizadas
origem = '/content/images_cut'

# Mover imagens com base no nome
for nome_arquivo in os.listdir(origem):
    caminho_origem = os.path.join(origem, nome_arquivo)

    if 'carie' in nome_arquivo.lower():
        shutil.move(caminho_origem, '/content/images/carie/' + nome_arquivo)
    elif 'saudavel' in nome_arquivo.lower():
        shutil.move(caminho_origem, '/content/images/saudavel/' + nome_arquivo)

print("✅ Imagens organizadas com sucesso!")


✅ Imagens organizadas com sucesso!


In [22]:
import os
import shutil

# Lista todas as imagens
arquivos = sorted(os.listdir('/content/images_cut'))

# Dividir ao meio: metade carie, metade saudavel (exemplo com 50%)
meio = len(arquivos) // 2
carie_imgs = arquivos[:meio]
saudavel_imgs = arquivos[meio:]

# Criar pastas destino
os.makedirs('/content/images/carie', exist_ok=True)
os.makedirs('/content/images/saudavel', exist_ok=True)

# Mover para carie
for nome in carie_imgs:
    shutil.move(f'/content/images_cut/{nome}', f'/content/images/carie/{nome}')

# Mover para saudavel
for nome in saudavel_imgs:
    shutil.move(f'/content/images_cut/{nome}', f'/content/images/saudavel/{nome}')

print("✅ Imagens distribuídas entre carie e saudavel.")


✅ Imagens distribuídas entre carie e saudavel.


## 6. Carregamento do Conjunto de Dados

Verificamos o diretório onde os dados foram extraídos e listamos as classes (subpastas) existentes. Também podemos criar variáveis para armazenar o caminho principal dos dados.


In [23]:
import os
import pandas as pd

data_dir = '/content/images'

classes = os.listdir(data_dir)
num_classes = len(classes)

print(f"Classes encontradas: {classes}")
print(f"Número de classes: {num_classes}")


Classes encontradas: ['856.png', '935.png', '885.png', '939.png', '866.png', '842.png', '378.png', '702.png', '872.png', '372.png', '365.png', '1088.png', '736.png', '710.png', '777.png', '376.png', '822.png', '724.png', '759.png', '708.png', '392.png', '783.png', '857.png', '1067.png', '758.png', '996.png', '861.png', '360.png', '954.png', '790.png', '818.png', '310.png', '388.png', '740.png', '369.png', '886.png', '1016.png', '367.png', '714.png', '867.png', '1058.png', '913.png', '761.png', '855.png', '732.png', '1026.png', '858.png', '769.png', '306.png', '370.png', '793.png', '859.png', '1080.png', '323.png', '812.png', '875.png', '981.png', '837.png', '354.png', '748.png', '1050.png', '883.png', '755.png', '350.png', '1018.png', '396.png', '1091.png', '810.png', '901.png', '1009.png', '395.png', '347.png', '934.png', 'carie', '1033.png', '784.png', '715.png', '1042.png', '979.png', '1008.png', '1096.png', '1092.png', 'saudavel', '750.png', '778.png', '963.png', '925.png', '819.pn

## 7. Pré-Processamento de Dados

Nesta etapa, utilizamos o `ImageDataGenerator` para:
- Redimensionar os valores de pixel das imagens (dividindo por 255).
- Separar o dataset em treinamento e validação (usando `validation_split=0.2`).
- Ajustar o tamanho das imagens (150x150) e o batch size (32).

**Por que isso é importante?**
- Redimensionar (rescale) ajuda a normalizar os dados.
- Dividir em treino/validação permite monitorar se há overfitting.
- Padronizar tamanho de imagem facilita o processamento pela rede neural.


In [29]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_dir = '/content/images'

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 80 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


## 8. Construção do Modelo

Utilizamos uma rede neural **convolucional** simples, com:
- Camadas de `Conv2D` para extração de características.
- Camadas de `MaxPooling2D` para redução da dimensionalidade.
- `Flatten` para achatar a saída das convoluções.
- `Dense` para as camadas totalmente conectadas.
- `Dropout` para evitar overfitting.
- `softmax` na camada final, pois há mais de uma classe (multiclasse).

**Como isso melhora o modelo?**  
- As camadas convolucionais ajudam a detectar padrões em imagens (bordas, texturas).  
- O pooling reduz a complexidade, mantendo as informações mais relevantes.  
- O Dropout reduz o risco de o modelo decorar o conjunto de treinamento.


In [30]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

num_classes = train_generator.num_classes  # Garante que seja 2 automaticamente

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Corrigido: saída com número real de classes
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()



Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 82944)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │    10,616,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,636,610 (40.58 MB)

 Trainable params: 10,636,610 (40.58 MB)

 Non-trainable params: 0 (0.00 B)

## 9. Compilação do Modelo

Aqui definimos:
- Otimizador: `adam` (geralmente um bom ponto de partida).
- Função de perda: `categorical_crossentropy` (pois temos mais de 2 classes).
- Métrica principal: `accuracy`.

**Observação**: Poderíamos testar outros otimizadores ou ajustar a taxa de aprendizado para ver se obtemos melhores resultados.


In [26]:
from PIL import Image
import os

for pasta in ['carie', 'saudavel']:
    caminho_pasta = f'/content/images/{pasta}'
    for arquivo in os.listdir(caminho_pasta):
        caminho_arquivo = os.path.join(caminho_pasta, arquivo)
        try:
            img = Image.open(caminho_arquivo)
            img.verify()  # Verifica integridade
        except Exception as e:
            print(f"⚠️ Erro ao abrir {arquivo}: {e}")


In [28]:
print("Classes detectadas:", train_generator.class_indices)
print("Quantidade de treino:", train_generator.samples)
print("Quantidade de validação:", validation_generator.samples)


Classes detectadas: {'carie': 0, 'saudavel': 1}
Quantidade de treino: 80
Quantidade de validação: 20


In [31]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.5195 - loss: 2.3118 - val_accuracy: 0.5000 - val_loss: 3.9321
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.5573 - loss: 2.9051 - val_accuracy: 0.5000 - val_loss: 1.0837
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 4s/step - accuracy: 0.5273 - loss: 1.0307 - val_accuracy: 0.5000 - val_loss: 0.6959
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.4854 - loss: 0.7140 - val_accuracy: 0.5000 - val_loss: 0.6890
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.6172 - loss: 0.6809 - val_accuracy: 0.6000 - val_loss: 0.6807
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.5547 - loss: 0.6748 - val_accuracy: 0.6500 - val_loss: 0.6764
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.5646 - loss: 0.6651 - val_accuracy: 0.6000 - val_loss: 0.6687
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.6365 - loss: 0.6124 - val_accuracy: 0.5500 - val_loss: 0.6797
Epoch 9/10
3/3 

## 10. Treinamento do Modelo

Treinamos o modelo usando:
- `train_generator` para o conjunto de treino.
- `validation_generator` para o conjunto de validação.
- `epochs=10` (pode ajustar conforme necessidade).

**Melhorias**:
- Podemos incluir callbacks como `EarlyStopping` ou `ModelCheckpoint` para interromper o treinamento quando não houver melhora.
- Monitorar métricas como acurácia e perda ao longo das épocas.


In [32]:
loss, accuracy = model.evaluate(validation_generator)
print(f"Acurácia: {accuracy * 100:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 731ms/step - accuracy: 0.6000 - loss: 0.6547
Acurácia: 60.00%


## 11. Avaliação do Modelo

Aqui avaliamos o modelo usando o `validation_generator`. Também podemos calcular a acurácia, que indica o percentual de acertos na classificação.


In [ ]:

loss, accuracy = model.evaluate(validation_generator)
print(f'Acurácia: {accuracy * 100:.2f}%')

3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 671ms/step - accuracy: 1.0000 - loss: 0.0144
Acurácia: 100.00%


In [33]:
model.save('modelo_caries.h5')
print("Modelo salvo com sucesso!")


Modelo salvo com sucesso!
